In [2]:
import numpy as np 
import cv2
import os

In [63]:
original_image_name = '8.png'
image = cv2.imread(f'gray_{original_image_name}', cv2.IMREAD_GRAYSCALE)
orginal_image = cv2.imread(original_image_name, cv2.IMREAD_ANYDEPTH)
print(orginal_image.shape)

row_max_values = np.max(image, axis=1)
row_max_values

row_max_indices = np.argmax(image, axis=1)
row_max_indices = row_max_indices.tolist()

(10000, 4008)


In [64]:
row_max_indices

[1930,
 1932,
 1931,
 1932,
 1919,
 1934,
 1934,
 1933,
 1935,
 1935,
 1935,
 1932,
 1932,
 1928,
 1939,
 1927,
 1928,
 1928,
 1927,
 1925,
 1939,
 1935,
 1926,
 1928,
 1930,
 1928,
 1930,
 1930,
 1937,
 1932,
 1934,
 1936,
 1936,
 1937,
 1936,
 1935,
 1937,
 1944,
 1925,
 1924,
 1924,
 1926,
 1925,
 1942,
 1932,
 1933,
 1934,
 1936,
 1942,
 1931,
 1932,
 1927,
 1925,
 1924,
 1925,
 1925,
 1936,
 1928,
 1927,
 1938,
 1929,
 1951,
 1921,
 1923,
 1922,
 1949,
 1932,
 1933,
 1930,
 1931,
 1950,
 1931,
 1930,
 1925,
 1933,
 1927,
 1944,
 1957,
 1939,
 1943,
 1953,
 1954,
 1952,
 1935,
 1927,
 1933,
 1935,
 1933,
 1924,
 1924,
 1945,
 1944,
 1943,
 1943,
 1912,
 1912,
 1939,
 1938,
 1938,
 1939,
 1939,
 1937,
 1933,
 1944,
 1930,
 1929,
 1928,
 1928,
 1928,
 1926,
 1926,
 1931,
 1924,
 1925,
 1942,
 1929,
 1928,
 1927,
 1927,
 1928,
 1929,
 1930,
 1926,
 1928,
 1925,
 1923,
 1924,
 1925,
 1926,
 1927,
 1928,
 1931,
 1960,
 1931,
 1930,
 1930,
 1920,
 1919,
 1921,
 1923,
 1925,
 1925,
 1939,

In [75]:
# orginal_image[:, row_max_indices].shape
heights = []
widths = []
avg_row_indice = np.average(row_max_indices)
for i in range(len(row_max_indices)):
    row = orginal_image[i, :]
    # filter row value is zero
    max_height_idx = row_max_indices[i]
    max_height = row[max_height_idx]
    # row_filter = row[row > 0]
    # print(f'max height {max_height}')
    left_avg = np.average(row[max_height_idx+150:max_height_idx+250])
    right_avg = np.average(row[max_height_idx-250:max_height_idx-150])
    if i == 8231:
        print("....")
    
    if right_avg < 10000 or left_avg < 10000 or max_height_idx < 100:
        # handle edge case
        if max_height_idx < avg_row_indice:
            # left part
            row = row[max_height_idx+100:]
            max_height_idx = np.argmax(row)
            if max_height_idx > 100:
                max_height = row[max_height_idx]
                left_avg = np.average(row[max_height_idx+150:max_height_idx+250])
                right_avg = np.average(row[max(max_height_idx-250,0): max(max_height_idx-150,max_height_idx+1)])
            else:
                # maybe a hole
                row_filter = row[row > 0][50:-50]
                max_height_idx = np.argmin(row_filter)
                max_height = row[max_height_idx]
                left_avg = np.average(row[max_height_idx+150:max_height_idx+250])
                right_avg = np.average(row[max(max_height_idx-250,0): max(max_height_idx-150,max_height_idx+1)])
            ...
        if max_height_idx > avg_row_indice:
            # right part
            ...
        ...
    paine_avg = (left_avg + right_avg) / 2
    # print(f'left avg {left_avg} right avg {right_avg} max_height {max_height}')
    if i == 8231:
        print("....")
    heights.append(abs(max_height - paine_avg))
    
    if max_height - paine_avg > 800:
        ...
        # print(f'max_height_idx {max_height_idx}')
        ...
    
    # iter max_height_idx to get idx lower than paine_avg
    for j in range(max_height_idx, 0, -1):
        if row[j] <= right_avg:
            # print(f'left {j}')
            break
    left_j = j
    for j in range(max_height_idx, len(row)):
        if row[j] <= left_avg:
            # print(f'right {j}')
            break
    right_j = j
    left_dis = abs(max_height_idx - left_j)
    right_dis = abs(max_height_idx - right_j)
    
    dis = right_j - left_j
    if dis > 150:
        if left_dis > right_dis:
            dis = right_dis * 2
        else:
            dis = left_dis * 2
        # print(f'max_height_idx {max_height_idx} left_j {left_j} right_j {right_j}')
        # print(".....")
    widths.append(dis)
    # print(f'dis {dis}')
    # if i > 1:
    # break
    
print(f'avg height {np.average(heights)}, min height {np.min(heights)}, max height {np.max(heights)}')
print(f'avg width {np.average(widths)}, min width {np.min(widths)}, max width {np.max(widths)}')
    

....
....
avg height 379.1792195246701, min height 49.51000000000204, max height 1082.5550000000003
avg width 142.2618, min width 0, max width 226


In [27]:
#calc circle

import cv2
import numpy as np

# 1. 读取图像
image = cv2.imread('C:/dev/cv_tools_box/3d/circle.bmp', cv2.IMREAD_COLOR)
image = cv2.GaussianBlur(image,(3, 3),0)

scale = 1
image = cv2.resize(image, (0, 0), fx=1/scale, fy=1/scale)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# _, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)


# print(gray.shape)
# ccircle = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, int(2000/scale), param1=253, param2=21, minRadius=int(1025.0/scale), maxRadius=int(1150/scale))
# ccircle = np.uint16(np.around(ccircle))
# for i in ccircle[0, :]:
#     cv2.circle(image, (int(i[0]), int(i[1])), 1, (255, 0, 0), 3)  # 画圆心
#     cv2.circle(image, (int(i[0]), int(i[1])), int(i[2]), (0, 0, 255), 3)  # 在原图上画圆，圆心，半径，颜色，线框
    
ccircle = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, int(5000/scale), param1=253, param2=21, minRadius=int(1150/scale), maxRadius=int(1200/scale))
ccircle = np.uint16(np.around(ccircle))
for i in ccircle[0, :]:
    cv2.circle(image, (int(i[0]), int(i[1])), 1, (255, 0, 0), 3)  # 画圆心
    cv2.circle(image, (int(i[0]), int(i[1])), int(i[2]), (0, 0, 255), 3)  # 在原图上画圆，圆心，半径，颜色，线框
   
cv2.imwrite('C:/dev/cv_tools_box/3d/circle.jpg', image)

True